# Setting Up the Environment

In [32]:
!pip install -r data/requirements.txt

  Obtaining dependency information for langchain==0.0.6 from https://files.pythonhosted.org/packages/a8/65/bb4dd9197c1212b5ce9690aff1e0b1f6f440014598763b82cc6ab7945be5/langchain-0.0.6-py3-none-any.whl.metadata
  Using cached langchain-0.0.6-py3-none-any.whl.metadata (6.5 kB)
Using cached langchain-0.0.6-py3-none-any.whl (60 kB)
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.354
    Uninstalling langchain-0.0.354:
      Successfully uninstalled langchain-0.0.354


In [33]:
import os
# used to load the pdfs 
# from openai import OpenAI
from langchain.document_loaders import PyPDFLoader

# used to create the retriever
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# used to create the retrieval tool
from langchain.agents import tool

# used to create the memory
from langchain.memory import ConversationBufferMemory

# used to create the prompt template
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.agents.openai_functions_agent.base import create_openai_functions_agent
from langchain.schema import SystemMessage
from langchain.prompts import MessagesPlaceholder

# used to create the agent executor
from langchain_openai import OpenAI
from langchain.agents import AgentExecutor

from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Optional, Type, List, Union
from langchain.prompts import PromptTemplate
from langchain.chains import LLMMathChain, LLMChain
from langchain.callbacks.manager import (AsyncCallbackManagerForToolRun, CallbackManagerForToolRun)
from langchain.memory import ConversationBufferWindowMemory
from langchain.agents import AgentType, initialize_agent, Tool, AgentExecutor, AgentOutputParser
from langchain.schema import AgentAction, AgentFinish, OutputParserException

from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent

# THis is the serp API wrapper than will help us route ambiguous queries to google search engine
from langchain_community.utilities import SerpAPIWrapper
from langchain.agents import load_tools

In [34]:
# insert your API key here
openai_api_key = open("data/API_key.txt").read().strip()
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["SERPAPI_API_KEY"] = '4bde37c74068633153b825cb3ff392ee3a6697a630674d5abb5d55be50f58a49'
memory_key = "history"

In [35]:
llm = ChatOpenAI(temperature = 0, openai_api_key=openai_api_key, model="gpt-3.5-turbo")

# Setting Up the RAG Agent

In [36]:
# Setting up the document loader. This returns the document in a page by page format wrapped in the pyDf wrapper
def load_document(doc_path=""):
    loader = PyPDFLoader(doc_path)
    doc = loader.load()
    
    return doc

In [16]:
def establish_retriever(doc_path, chunk_size=1000, overlap=50):
    """
        Recursive splitter tries to go to our specified chunksize by recursively splitting the document on ["\n\n", "\n", " ", ""]
        If the first split by \n\n is still large then it moves to the next character which is \n and tries to split by it.
        If it is still larger than our specified chunk size it moves to the next character.
        We fetch these chunks and then convert them into vector embeddings using OpenAi's embeddings.
        We store these embeddings into our vector database.
        We return the vector database object.
    """
    doc = load_document(doc_path)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=overlap)
    texts = text_splitter.split_documents(doc)
    
    embeddings = OpenAIEmbeddings()# creating embeddings
    db = FAISS.from_documents(texts, embeddings) # store the embeddings in our FAISS database
    retriever = db.as_retriever()
    
    return retriever

In [17]:
doc_path = "data/google_earning_call.pdf" # This is just an example, it can be anything
retriever = establish_retriever(doc_path)

# Creating Tools

In [18]:
# This is the retrieval tool that searches the documents and fetches chunks most similar to user query using FAISS.
@tool
def retriever_tool(query):
    "Searches and returns relevant documents to user queries regarding the uploaded document"
    docs = retriever.get_relevant_documents(query)
    return docs

tools = [retriever_tool]

In [19]:
# This is an arithmetic tool which simply performs any calculation the user might want ot perform. 
@tool 
def math_tool(query):
    "Takes in a arithmetic equation as query and solves the equation in a step-by-step manner."
    llm_math_chain = LLMMathChain.from_llm(llm, verbose=True)
    response = llm_math_chain.run(query)
    
    return response

tools.append(math_tool)

In [20]:
# This is the tool that searches the web for answers to out-of-knowledge queries 
serp_tool = load_tools(['serpapi'])
tools.append(*serp_tool)

# Defining the Agent

In [21]:
# Vital to retain memory since that is what enables the back-forth convo
memory = ConversationBufferMemory(memory_key=memory_key, return_messages=True)

In [22]:
# Our custom system prompt
sys_prompt = """
You are a helpful AI agent that tries it's best to provide the most relevant response to a user query. \
You have multiple custom tools at your disposal. Here are the roles of your 3 tools:\
1. retriever_tool: utilise this tool whenever the user asks you question related to a specific document. Reply with \
"I DO NOT KNOW. DO YOU WISH ME TO SEARCH THE WEB?" in case you cannot find answer to a user query from the documents retrieved by this tool.
2. math_tool: Utilise this function whenever asked a arithmetic related question.
3. serpapi_tool: use this tool whenever the retriever tool fails to answer the query. Ask the user before using this tool.\
If user says "yes search the web", then utilise this tool to answer the initial user query. 

You can also use these tools in a sequential manner to do more complex tasks. For example, a user might first ask\
you to fetch some number using the retrieval/serp_api tool and then ask you to do some arithmetic which you should\
route to the math_tool.

Finally, follow these instructions for all generations:
1. Be concise. 
2. Be precise with response. 
3. Try to follow the mentioned instructions at all times.
4. Structure your response so that it is easy for the user to read.
5. Follow this heirarchy when using your tools:
When asked a question which involves retrieval and arithmetic, first call the retrieval tool to fetch the query specific data. The run this data through the math_tool and then return the response. 
When asked any other question, invoke the retrieval tool first. If no relevant response is found, return "I DO NOT KNOW. WANT ME TO SEARCH THE WEB?".
"""

In [23]:
# Here we are simply wrapping the prompts to feed into the model
system_message = SystemMessage(
        content=(f"{sys_prompt}")
)
prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[MessagesPlaceholder(variable_name=memory_key)]
)

In [24]:
# Now we define the AGENT
agent = create_openai_functions_agent(llm=llm, tools=tools, prompt=prompt)

In [25]:
# This is where everything is wrapped together to form our RAG agent. RAG is just one capability enabled by our retrieval tool. Adding more tools gives more abilities
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)

In [26]:
# This function passes user queries to the model and returns the inferred response
def infer(query):
    agent_input = {'input':query}
    result = agent_executor(agent_input)
    
    return result['output']

# Example Inferences

In [27]:
# Instance of the retriver tool being called.
print(infer("Can you tell me what was the effect of staff cuts on Google in terms of severance costs?"))



> Entering new AgentExecutor chain...

Invoking: `retriever_tool` with `{'query': 'effect of staff cuts on Google severance costs'}`


[Document(page_content='Severance and office space exit charges are included within our consolidated statements of income as follows (in \nmillions):\nQuarter Ended December 31, 2023 Year Ended December 31, 2023\nSeverance \nand Related Office Space TotalSeverance \nand Related Office Space Total\nCost of revenues $ 2 $ 235 $ 237 $ 479 $ 481 $ 960 \nResearch and development  1  602  603 848  870  1,718 \nSales and marketing  3  198  201 497  257  754 \nGeneral and administrative  1  161  162 264  237  501 \nTotal charges $ 7 $ 1,196 $ 1,203 $ 2,088 $ 1,845 $ 3,933 \nFor segment reporting, the substantial majority of these charges are included within Alphabet-level activities in our \nsegment result s.\nChange in Useful Lives of Our Server and Network Equipment\nIn January 2023, we completed an assessment of the useful lives of our servers and network 

In [28]:
print(infer("What was the operating income in 2022 and in 2023? What was the percentage change betweeen these operating incomes"))



> Entering new AgentExecutor chain...

Invoking: `retriever_tool` with `{'query': 'Google operating income 2022 2023'}`


[Document(page_content='Q4 2023  Supplemental Information  (in millions, except for number of employees; unaudited)\nRevenues, Traffic Acquisition Costs (TAC), and Number of Employees\nQuarter Ended December 31,\n2022 2023\nGoogle Search & other $ 42,604 $ 48,020 \nYouTube ads  7,963  9,200 \nGoogle Network  8,475  8,297 \nGoogle advertising  59,042  65,517 \nGoogle subscriptions, platforms, and devices(1) 8,796  10,794 \nGoogle Services total  67,838  76,311 \nGoogle Cloud  7,315  9,192 \nOther Bets  226  657 \nHedging gains (losses)  669  150 \nTotal revenues $ 76,048 $ 86,310 \nTotal TAC $ 12,925 $ 13,986 \nNumber of employees  190,234  182,502 \n(1) Formerly “Google other.”\nSegment Operating Results\nQuarter Ended December 31,\n2022 2023\nOperating income (loss):\nGoogle Services $ 20,222 $ 26,730 \nGoogle Cloud  (186)  864 \nOther Bets  (1,237)  (863) \nAlph

In [29]:
# An instance of when out-of-knowledge question has been asked. Look at how the model obeys the system prompt!
print(infer("who is the president of pakistan?"))



> Entering new AgentExecutor chain...

Invoking: `retriever_tool` with `{'query': 'president of Pakistan'}`


[Document(page_content='Alphabet Announces Fourth Quarter and Fiscal Year  2023  Results\nMOUNTAIN VIEW, Calif. – January 30, 2024  – Alphabet Inc. (NASDAQ: GOOG, GOOGL) today announced \nfinancial results for the quarter and fiscal year ended  December 31, 2023 .\nSundar Pichai, CEO, said: “We are pleased with the ongoing strength in Search and the growing contribution from \nYouTube and Cloud. Each of these is already benefiting from our AI investments and innovation. As we enter the \nGemini era, the best is yet to come.”\nRuth Porat, President and Chief Investment Officer; CFO said: “We ended 2023 with very strong fourth quarter \nfinancial results, with Q4 consolidated revenues of $86 billion, up 13% year over year. We remain committed to our \nwork to durably re-engineer our cost base as we invest to support our growth opportunities.”\nQ4 2023  Financial Highlights\nThe